In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader

#input transformation seqeunce
transform = transforms.Compose([
        transforms.Resize((28, 28)), #resize input to 28x28
        transforms.ToTensor(), #convert image to pytorch tensor [0, 1] range
        transforms.Normalize((0.5, ), (0.5, )) #normalize values to [-1, 1] range
])

#load traning and test FashionMNIST dataset
'''
root --> save location if download is needed#train --> what portion of the dataset to load (training or testing)
download --> download dataset if not present in root
transform --> apply transitions to each image as it is being accessed
'''
full_train_dataset = torchvision.datasets.FashionMNIST(
        root='./data', train=True, download=True, transform=transform
)

test_dataset = torchvision.datasets.FashionMNIST(
        root='./data', train=False, download=True, transform=transform
)

#Split training dataset into training and validation sets (80%/20%)
train_size = int(0.8 * len(full_train_dataset))
val_size = int(0.2 * len(full_train_dataset))
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

#Create DataLoaders
'''
batch_size --> each batch retreived by the loader will get 512 samples at once
        - reduces gradient noise
        - allows parallel computing by GPU
        - forward pass --> CEL --> backward pass --> Gradient update
shuffle --> shuffle order samples are loaded
        - shuffle for training set to prevent overfitting
        - no shuffle for validation and testing set to ensure consistent & reproducible evaluation
num_workers --> how many parallel processes will be handling data loading
'''
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=2)
test = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=2)

print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}")



100.0%
100.0%
100.0%
100.0%

Train: 48000 | Val: 12000 | Test: 10000


In [ ]:
import torch.nn.functional as F

class FashionMNISTNet(nn.Module):
        '''
        input: 28x28
        conv1 = 32x28x28
        maxpool = 32x14x14
        conv2 = 64x16x16
        maxpool = 64x8x8
        conv3 = 64x8x8
        maxpool = 64x4x4
        flatten = 1024
        '''
        def __init__(self):
                super(FashionMNISTNet, self).__init__()
                self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
                self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=2)
                self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
                self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
                self.fc1 = nn.Linear(in_features=1024, out_features=256)
                self.fc2 = nn.Linear(in_features=256, out_features=128)
                self.fc3 = nn.Linear(in_features=128, out_features=10)

        def forward(self, x):
                x = self.pool(F.relu(self.conv1(x)))
                x = self.pool(F.relu(self.conv2(x)))
                x = self.pool(F.relu(self.conv3(x)))
                x = torch.flatten(x, start_dim=1)
                x = F.relu(self.fc1(x))
                x = F.relu(self.fc2(x))
                x = F.relu(self.fc3(x))
                x = F.softmax(x, dim=1)
                return x
        
net = FashionMNISTNet()
print(net)

FashionMNISTNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)
